In [19]:
from os.path import exists
from Bio import SeqIO
import pandas as pd
import re

class flu_mutation:
    def __init__(self,
        position,
        mutation,
        strain1,
        strain2):
        assert (type(position) == str), "Position must be a string"
        assert (type(mutation) == str), "Mutation must be identified as a string"
        assert (type(strain1) == str), "Strain 1 must be a string identifier"
        assert (type(strain2) == str), "Strain 2 must be a string identifier"
        self.position = position
        self.mutation = mutation
        self.strain1 = strain1
        self.strain2 = strain2
    def __str__(self):
        return f"Strain 1: {self.strain1}, Strain 2: {self.strain2}, Mutation: {self.mutation}"


class flu_seq:
    def __init__(self,
        name,
        lineage,
        query_sequence_file,
        query_sequence_id,
        is_aligned,
        position_map):
        assert (exists(query_sequence_file)), "Query sequence file must exist"
        assert (type(lineage) == str), "Lineage must be a string."
        assert (type(name) == str), "Name must be a string."
        assert (type(query_sequence_id) == str), "Query sequence ID must be a string."
        assert (type(is_aligned) == bool), "is_aligned must be a boolean."
        query_seqs = {s.id: s for s in SeqIO.parse(query_sequence_file, "fasta")}
        assert (len(query_seqs) >= 1), "Query sequence file must contain at least 1 sequence."
        self.name = name
        self.lineage = lineage
        self.sequence = query_seqs[query_sequence_id]
        self.is_aligned = is_aligned
        self.position_map = position_map
        self.query_sequence_file = query_sequence_file

        # Get PNGS sites
        gly = re.compile("N[-]*[A-O,Q-Z][-]*[S,T]")
        self.pngs = [position_map.loc[m.start(), "H3"] for m in gly.finditer(str(self.sequence.seq))]

class seq_compare:
    def __init__(self, seq1, seq2):
        assert (type(seq1) == flu_seq), "Seq1 must be a flu_seq object"
        assert (type(seq2) == flu_seq), "Seq2 must be a flu_seq object"
        assert (seq1.is_aligned), "Seq1 must be aligned"
        assert (seq2.is_aligned), "Seq2 must be aligned"
        assert (seq1.query_sequence_file == seq2.query_sequence_file), "Sequences must be from the same alignment"
        self.seq1 = seq1
        self.seq2 = seq2
    def identify_mutations(self):
        mutations_out = []
        for i, (b1, b2) in enumerate(zip(self.seq1.sequence.seq, self.seq2.sequence.seq)):
            if b1 != b2:
                assert (self.seq1.position_map.loc[i, "H3"] == self.seq2.position_map.loc[i, "H3"]), "Sequences must use same position map"
                p = self.seq1.position_map.loc[i, "H3"]
                mutations_out.append(
                    flu_mutation(position = str(p),
                        mutation = "".join([str(b1), str(p), str(b2)]),
                        strain1 = self.seq1.name,
                        strain2 = self.seq2.name)
                )
        return mutations_out
    def identify_PNGS_changes(self):
        glycan_deletions = set(self.seq1.pngs) - set(self.seq2.pngs)
        glycan_additions = set(self.seq2.pngs) - set(self.seq1.pngs)
        glycans_shared = set(self.seq2.pngs).intersection(set(self.seq1.pngs))
        return {"glycan_deletions": glycan_deletions, "glycan_additions": glycan_additions, "glycans_shared": glycans_shared}  

In [20]:
position_map_infile = "position_map_infile.csv"
aligned_seqs = "./H3N2_HA_GISAID.fasta"
q1_id = "EPI1487157"
q1_name = "A/Minnesota/41/2019"
q2_id = "EPI1752480"
q2_name = "A/Tasmania/503/2020"
seq_lineage = "H3N2"

position_map = pd.read_csv("EPI1487157_H3_Conversion.txt", sep = "\t")
position_map = position_map[position_map.Query != "-"].reset_index()

s1 = flu_seq(name = q1_name,
    lineage = seq_lineage,
    query_sequence_file = aligned_seqs,
    query_sequence_id = q1_id,
    is_aligned = True,
    position_map = position_map
    )

s2 = flu_seq(name = q2_name,
    lineage = seq_lineage,
    query_sequence_file = aligned_seqs,
    query_sequence_id = q2_id,
    is_aligned = True,
    position_map = position_map
    )

mutation_list = seq_compare(seq1 = s1, seq2 = s2).identify_mutations()

In [21]:
seq_compare(seq1 = s1, seq2 = s2).identify_PNGS_changes()

{'glycan_deletions': set(),
 'glycan_additions': {'126', '133'},
 'glycans_shared': {'122',
  '158',
  '165',
  '22',
  '246',
  '285',
  '38',
  '45',
  '483',
  '63',
  '8'}}

In [11]:
from pymol import cmd, movie

cmd.reinitialize()
cmd.fetch('4we8')
cmd.symexp('sym','4we8','4we8','3')
cmd.delete('sym11000000')
cmd.delete('sym04000000')
cmd.center('4we8, sym01000000, sym02000000')
cmd.set('ray_trace_mode', 0)

cmd.select('glycans', 'resn NAG or resn BMA or resn MAN')
cmd.hide('everything')
cmd.show('spheres')
cmd.show('sticks', 'glycans')
cmd.hide('spheres', 'glycans')
cmd.remove('solvent')
cmd.color('grey', 'all')


In [14]:
[m.position for m in mutation_list if m.position != "-"]

['83',
 '94',
 '128',
 '131',
 '135',
 '137',
 '138',
 '186',
 '195',
 '198',
 '522',
 '529']

In [22]:
for m in mutation_list:
    print(m.mutation)

T-A
K83E
Y94N
A128T
T131K
K135T
F137S
S138A
G186S
Y195F
S198P
I522M
V529I
